In [110]:
import pandas as pd
import json
from collections import defaultdict
import re
import dataExtraction2
import dataExtraction3
import dataExtraction4

#### Con este bucle obtenemos los datos sobre la inicialización de todos los niveles jugados. PRUEBA

Si se han usado los pasos minimos:

In [265]:
def porDefecto():
    return defaultdict(lambda: {"tarjetasCreadas" : 0})

def extraerDatosTarjetas(rawData):
    erGameObject = re.compile(r'\bgame-object$\b')
    erInteracted = re.compile(r'\binteracted$\b')
    erCompleted = re.compile(r'\bcompleted$\b')
    erInitialized = re.compile(r'\binitialized$\b')
    erLevel = re.compile(r'\blevel$\b')
    erIdLevel = re.compile(r'/')
    
    data = defaultdict(porDefecto)
    dataCreadas = defaultdict(porDefecto)
    dataMovidas = defaultdict(porDefecto)
    dataEliminadas = defaultdict(porDefecto)
    dataAttach = defaultdict(porDefecto)
    dataDettach = defaultdict(porDefecto)
    dataDuplicate = defaultdict(porDefecto)
    dataDeclare = defaultdict(porDefecto)

    nivel = defaultdict()

    for evento in rawData:
        
        verb = evento["verb"]["id"]
        obj = evento["object"]["definition"]["type"]
        levelCode = 0
        eventos = [""]

        #Empezamos nivel y lo guardamos
        if erLevel.search(obj) and erInitialized.search(verb):
            if "result" in evento:
                name = evento["actor"]["name"]
                levelCode = erIdLevel.split(evento["object"]["id"])[-1]
                nivel[name] = levelCode

        elif erGameObject.search(obj) : #Si el objeto de la acción es un objecto
            name = evento["actor"]["name"]

            if erInteracted.search(verb):
                    try:
                        if nivel[name] != "editor_level":
                            #print(evento)
                            bloque = evento["result"]["extensions"]["block_type"]
                            #dataExtraction2.anadirAlDiccionario(data, evento, name, nivel)
                            if evento["result"]["extensions"]["action"] == "create":
                                if evento["result"]["extensions"]["block_type"] in dataCreadas[name][levelCode]:
                                        creadas = int(dataCreadas[name][levelCode][bloque][0]["tarjetasCreadas"])
                                        dataCreadas[name][levelCode][bloque][0]["tarjetasCreadas"] = creadas + 1
                                else:
                                    dataCreadas[name][levelCode][bloque] = [{"tarjetasCreadas" : 1}]

                            elif evento["result"]["extensions"]["action"] == "move":
                                if evento["result"]["extensions"]["block_type"] in dataMovidas[name][levelCode]:
                                        movidas = int(dataMovidas[name][levelCode][bloque][0]["tarjetasMovidas"])
                                        dataMovidas[name][levelCode][bloque][0]["tarjetasMovidas"] = movidas + 1
                                else:
                                    dataMovidas[name][levelCode][bloque] = [{"tarjetasMovidas" : 1}]

                            elif evento["result"]["extensions"]["action"] == "remove":
                                if evento["result"]["extensions"]["block_type"] in dataEliminadas[name][levelCode]:
                                        eliminadas = int(dataEliminadas[name][levelCode][bloque][0]["tarjetasEliminadas"])
                                        dataEliminadas[name][levelCode][bloque][0]["tarjetasEliminadas"] = movidas + 1
                                else:
                                    dataEliminadas[name][levelCode][bloque] = [{"tarjetasEliminadas" : 1}]

                            elif evento["result"]["extensions"]["action"] == "attach" or evento["result"]["extensions"]["action"] == "attach_to_input" or evento["result"]["extensions"]["action"] == "attach_to_bottom" or evento["result"]["extensions"]["action"] == "attach_to_top":
                                if evento["result"]["extensions"]["block_type"] in dataAttach[name][levelCode]:
                                        encajadas = int(dataAttach[name][levelCode][bloque][0]["tarjetasEncajadas"])
                                        dataAttach[name][levelCode][bloque][0]["tarjetasEncajadas"] = encajadas + 1
                                else:
                                    dataAttach[name][levelCode][bloque] = [{"tarjetasEncajadas" : 1}]

                            elif evento["result"]["extensions"]["action"] == "dettach":
                                if evento["result"]["extensions"]["block_type"] in dataDettach[name][levelCode]:
                                        encajadas = int(dataDettach[name][levelCode][bloque][0]["tarjetasEncajadas"])
                                        dataDettach[name][levelCode][bloque][0]["tarjetasEncajadas"] = encajadas + 1
                                else:
                                    dataDettach[name][levelCode][bloque] = [{"tarjetasEncajadas" : 1}]

                            elif evento["result"]["extensions"]["action"] == "duplicate":
                                if evento["result"]["extensions"]["block_type"] in dataDuplicate[name][levelCode]:
                                        encajadas = int(dataEliminadas[name][levelCode][bloque][0]["tarjetasDuplicadas"])
                                        dataDuplicate[name][levelCode][bloque][0]["tarjetasDuplicadas"] = encajadas + 1
                                else:
                                    dataDuplicate[name][levelCode][bloque] = [{"tarjetasDuplicadas" : 1}]
                        
                            elif evento["result"]["extensions"]["action"] == "declare":
                                if evento["result"]["extensions"]["block_type"] in dataDeclare[name][levelCode]:
                                    encajadas = int(dataDeclare[name][levelCode][bloque][0]["tarjetasDeclaradas"])
                                    dataDeclare[name][levelCode][bloque][0]["tarjetasDeclaradas"] = encajadas + 1
                                else:
                                    dataDeclare[name][levelCode][bloque] = [{"tarjetasDeclaradas" : 1}]

                            else:
                                print("meeeu " + evento["result"]["extensions"]["action"])

                            #print(dataCreadas[name][levelCode])

                    except KeyError:
                        pass

    print(eventos)                

    return dataCreadas, dataMovidas, dataEliminadas, dataAttach, dataDettach, dataDuplicate, dataDeclare 

In [266]:
"""#Laseres
def porDefecto():
    return defaultdict(lambda: {"laserCreados" : 0, "laserMovidos" : 0, "laserEliminados" : 0})

def extraerDatosLaseres(rawData):
    erGameObject = re.compile(r'\bgame-object$\b')
    erInteracted = re.compile(r'\binteracted$\b')
    
    data = defaultdict(porDefecto)

    for evento in rawData:
        
        verb = evento["verb"]["id"]
        obj = evento["object"]["definition"]["type"]
        if erGameObject.search(obj) : #Si el objeto de la acción es un objecto
            name = evento["actor"]["name"]

            if erInteracted.search(verb):
                print(evento)
                try:
                    dataExtraction3.anadirAlDiccionario(data, evento, name)
                except KeyError:
                    pass

    return data"""

'#Laseres\ndef porDefecto():\n    return defaultdict(lambda: {"laserCreados" : 0, "laserMovidos" : 0, "laserEliminados" : 0})\n\ndef extraerDatosLaseres(rawData):\n    erGameObject = re.compile(r\'\x08game-object$\x08\')\n    erInteracted = re.compile(r\'\x08interacted$\x08\')\n    \n    data = defaultdict(porDefecto)\n\n    for evento in rawData:\n        \n        verb = evento["verb"]["id"]\n        obj = evento["object"]["definition"]["type"]\n        if erGameObject.search(obj) : #Si el objeto de la acción es un objecto\n            name = evento["actor"]["name"]\n\n            if erInteracted.search(verb):\n                print(evento)\n                try:\n                    dataExtraction3.anadirAlDiccionario(data, evento, name)\n                except KeyError:\n                    pass\n\n    return data'

In [267]:
"""#Retry button
def porDefecto():
    return defaultdict(lambda: {"botonRetryUsado" : 0})

def extraerDatosBoton(rawData):
    erGameObject = re.compile(r'\bgame-object$\b')
    erInteracted = re.compile(r'\binteracted$\b')
    erButton = re.compile(r'\bretry_button$\b')
    
    data = defaultdict(porDefecto)

    for evento in rawData:
        
        verb = evento["verb"]["id"]
        obj = evento["object"]["definition"]["type"]
        if erGameObject.search(obj) : #Si el objeto de la acción es un objecto
            name = evento["actor"]["name"]

            if erInteracted.search(verb):
                print(evento)
                if erButton.search(evento["object"]["id"]):
                    
                    try:
                        dataExtraction4.anadirAlDiccionario(data, evento, name)
                    except KeyError:
                        pass

    return data """

'#Retry button\ndef porDefecto():\n    return defaultdict(lambda: {"botonRetryUsado" : 0})\n\ndef extraerDatosBoton(rawData):\n    erGameObject = re.compile(r\'\x08game-object$\x08\')\n    erInteracted = re.compile(r\'\x08interacted$\x08\')\n    erButton = re.compile(r\'\x08retry_button$\x08\')\n    \n    data = defaultdict(porDefecto)\n\n    for evento in rawData:\n        \n        verb = evento["verb"]["id"]\n        obj = evento["object"]["definition"]["type"]\n        if erGameObject.search(obj) : #Si el objeto de la acción es un objecto\n            name = evento["actor"]["name"]\n\n            if erInteracted.search(verb):\n                print(evento)\n                if erButton.search(evento["object"]["id"]):\n                    \n                    try:\n                        dataExtraction4.anadirAlDiccionario(data, evento, name)\n                    except KeyError:\n                        pass\n\n    return data '

In [268]:
"""import pandas as pd
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
import json
from collections import defaultdict
import re
import dataExtraction2

def extraerDatosTarjetas(rawData):
    erGameObject = re.compile(r'\bgame-object$\b')
    erInteracted = re.compile(r'\binteracted$\b')
    
    data = defaultdict(porDefecto)

    for evento in rawData:
        verb = evento["verb"]["id"]
        obj = evento["object"]["definition"]["type"]
        if erGameObject.search(obj) : #Si el objeto de la acción es un objecto
            name = evento["actor"]["name"]

            if erInteracted.search(verb):
                try:
                    levelCode = evento["result"]["extensions"]["level"]
                    dataExtraction2.anadirAlDiccionario(data, evento, name, levelCode)
                except KeyError:
                    pass

    return data            

%matplotlib widget

JSONFile = open('traces_Articoding_Escolapias.json')
rawData = json.load(JSONFile)
data = extraerDatosTarjetas(rawData)

# Creamos o entorno
fig = plt.figure()  # Creamos unha figura (contenedor para elementos a debuxar)
ax = fig.add_subplot(projection='3d')  # Engadimos un Axes á figura (contén os elementos do debuxo)
plt.xlabel("attack")  # Nombramos os eixos
plt.ylabel("hp")

# Debuxamos o scatter
ax.scatter(data['gqoj'][0]["tarjetasCreadas"], data['gqoj'][0]["tarjetasMovidas"], data['gqoj'][0]["tarjetasEliminadas"])

plt.show()"""

'import pandas as pd\nimport matplotlib as mpl\nfrom matplotlib import cm\nimport matplotlib.pyplot as plt\nfrom mpl_toolkits import mplot3d\nfrom mpl_toolkits.mplot3d import Axes3D\nimport json\nfrom collections import defaultdict\nimport re\nimport dataExtraction2\n\ndef extraerDatosTarjetas(rawData):\n    erGameObject = re.compile(r\'\x08game-object$\x08\')\n    erInteracted = re.compile(r\'\x08interacted$\x08\')\n    \n    data = defaultdict(porDefecto)\n\n    for evento in rawData:\n        verb = evento["verb"]["id"]\n        obj = evento["object"]["definition"]["type"]\n        if erGameObject.search(obj) : #Si el objeto de la acción es un objecto\n            name = evento["actor"]["name"]\n\n            if erInteracted.search(verb):\n                try:\n                    levelCode = evento["result"]["extensions"]["level"]\n                    dataExtraction2.anadirAlDiccionario(data, evento, name, levelCode)\n                except KeyError:\n                    pass\n\n  

In [269]:
#%%
"""pokemon = pd.read_csv('pokemon.csv',  # Cargamos o csv
                       delimiter=',',
                       header=0)
pokemon.drop(['abilities','against_bug', 'against_dark', 'against_dragon', 'against_electric', 'against_fairy', 'against_fight', 'against_fire', 'against_flying', 'against_ghost', 'against_grass', 'against_ground', 'against_ice', 'against_normal', 'against_poison', 'against_psychic', 'against_rock', 'against_steel', 'against_water', 'base_egg_steps', 'base_happiness', 'base_total', 'capture_rate', 'classfication', 'experience_growth', 'height_m', 'japanese_name', 'name', 'percentage_male', 'pokedex_number', 'type1', 'type2', 'weight_kg', 'generation', 'sp_attack', 'sp_defense', 'defense', 'is_legendary'], axis = 'columns', inplace=True)

%matplotlib widget

# Creamos o entorno
fig = plt.figure()  # Creamos unha figura (contenedor para elementos a debuxar)
ax = fig.add_subplot(projection='3d')  # Engadimos un Axes á figura (contén os elementos do debuxo)
plt.xlabel("attack")  # Nombramos os eixos
plt.ylabel("hp")

# Debuxamos o scatter
ax.scatter(pokemon['attack'], pokemon['hp'], pokemon['speed'])

plt.show()"""


'pokemon = pd.read_csv(\'pokemon.csv\',  # Cargamos o csv\n                       delimiter=\',\',\n                       header=0)\npokemon.drop([\'abilities\',\'against_bug\', \'against_dark\', \'against_dragon\', \'against_electric\', \'against_fairy\', \'against_fight\', \'against_fire\', \'against_flying\', \'against_ghost\', \'against_grass\', \'against_ground\', \'against_ice\', \'against_normal\', \'against_poison\', \'against_psychic\', \'against_rock\', \'against_steel\', \'against_water\', \'base_egg_steps\', \'base_happiness\', \'base_total\', \'capture_rate\', \'classfication\', \'experience_growth\', \'height_m\', \'japanese_name\', \'name\', \'percentage_male\', \'pokedex_number\', \'type1\', \'type2\', \'weight_kg\', \'generation\', \'sp_attack\', \'sp_defense\', \'defense\', \'is_legendary\'], axis = \'columns\', inplace=True)\n\n%matplotlib widget\n\n# Creamos o entorno\nfig = plt.figure()  # Creamos unha figura (contenedor para elementos a debuxar)\nax = fig.add_sub

In [270]:
JSONFile = open('traces_Articoding_Escolapias.json')
rawData = json.load(JSONFile)
dataCreadas, dataMovidas, dataEliminadas, dataAttach, dataDettach = extraerDatosTarjetas(rawData)
#data = extraerDatosLaseres(rawData)
#data = extraerDatosBoton(rawData)
pd.DataFrame(dataCreadas)

JSONFile.close()

pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)
#pd.DataFrame(data)

pd.DataFrame(dataCreadas)

meeeu delete
meeeu delete
meeeu delete
meeeu delete
meeeu delete
meeeu create_clone
meeeu delete
meeeu delete
meeeu delete
meeeu delete
meeeu create_clone
meeeu create_clone
meeeu create_clone
meeeu delete
meeeu create_clone
meeeu create_clone
meeeu create_clone
meeeu create_clone
meeeu delete
meeeu create_clone
meeeu create_clone
meeeu create_clone
meeeu create_clone
meeeu create_clone
meeeu create_clone
meeeu delete
meeeu delete
meeeu delete
meeeu delete
meeeu delete
meeeu create_clone
meeeu delete
meeeu delete
meeeu delete
meeeu delete
meeeu delete
meeeu create_clone
meeeu delete
meeeu create_clone
meeeu delete
meeeu create_clone
meeeu create_clone
meeeu delete
meeeu create_clone
meeeu create_clone
meeeu create_clone
meeeu create_clone
meeeu create_clone
meeeu create_clone
meeeu create_clone
meeeu create_clone
meeeu create_clone
meeeu create_clone
meeeu create_clone
meeeu delete
meeeu create_clone
meeeu create_clone
meeeu delete
meeeu create_clone
meeeu create_clone
meeeu create_clo

ValueError: too many values to unpack (expected 5)

In [ ]:
pd.DataFrame(dataMovidas)

,gqoj,qeqyj,kmqou,loemd,gcpze,eprdu,bdwyj,zwzfv,vsxld,ipdcp,ikcar,srjon,ejeuw,khadt,gleqe,ftpnb,xnpqf,hjpbw,zxvkj,epqkm,sxipu,pbgky,uzzpo,fkmtp,pygoc,ubmtx,asopk,yvpmf,bhyhw,zcibf,krahz,oahey,kkoif,dunul,cmbfi,auvqa,onyep,rdiaf,grkij,tekwp,nmveq,tjfzy,accul,dwqwi,edxmq,yfohf,swvxx,sjpkc,dvmlu,gguuc,wjaxi,mjiso,tgwmn,hpdxj,ucdvy,ewqbs,mymho,psido,hqmhq,lwryl,kkliq,gdljy,fivin,yxewz,tinco,ztyed,camhi,fyyya,yxjji,zbgwk,rziia,tqgyp,ngavz,aotlm,qfsdj,mgaak,yazod,stiry,tavin,vddlg,atlzs,zdnso,lhrbj,vvqlm,jozyx,tvxow,jhbuv,wqmgq,knewj,huyyb,xtdrq,idmzi,rqlug,ahxym,pbwkf,itkas,gzgfk,buvgs,ptmkr,kzles,mewpz,thaho,fmtrq,qwnpf,pzfdd,icllt,rkmrw,yyvas,nplrf,uniee,rqaby,khfha,hcbyl,sbexy,xbmmw,ixrux,raqdt,pqwaf,qgcgc,tflzv,pleui,lohyo,fbldj,fmupf,eayrh,sybwc,rfofs,blvkx,prpej,bajbm,tcvyz,orjyn,cojqb,qbsjv,rsjxu,vkfnt,olugf,nhbnj,nuasx,esrit,epjcx
0,"{'tarjetasCreadas': 0, 'procedures_callnoreturn': [{'tarjetasMovidas': 2}], 'procedures_callreturn': [{'tarjetasMovidas': 1}], 'procedures_defreturn': [{'tarjetasMovidas': 2}], 'procedures_defnoreturn': [{'tarjetasMovidas': 1}], 'start_start': [{'tarjetasMovidas': 1}]}","{'tarjetasCreadas': 0, 'movement_move_laser': [{'tarjetasMovidas': 4}], 'variables_set': [{'tarjetasMovidas': 13}], 'variables_get': [{'tarjetasMovidas': 10}], 'math_number': [{'tarjetasMovidas': 2}], 'movement_rotate_laser': [{'tarjetasMovidas': 1}], 'text': [{'tarjetasMovidas': 5}], 'movement_rotate': [{'tarjetasMovidas': 1}], 'movement_move': [{'tarjetasMovidas': 1}], 'movement_activate_door': [{'tarjetasMovidas': 2}], 'logic_boolean': [{'tarjetasMovidas': 1}]}","{'tarjetasCreadas': 0, 'start_start': [{'tarjetasMovidas': 6}], 'variables_get': [{'tarjetasMovidas': 9}], 'math_number': [{'tarjetasMovidas': 7}], 'variables_set': [{'tarjetasMovidas': 7}], 'movement_move_laser': [{'tarjetasMovidas': 1}], 'text': [{'tarjetasMovidas': 1}], 'movement_rotate': [{'tarjetasMovidas': 1}]}","{'tarjetasCreadas': 0, 'variables_set': [{'tarjetasMovidas': 8}], 'math_number': [{'tarjetasMovidas': 3}], 'variables_get': [{'tarjetasMovidas': 11}], 'start_start': [{'tarjetasMovidas': 1}], 'movement_move_laser': [{'tarjetasMovidas': 1}], 'movement_rotate_laser': [{'tarjetasMovidas': 1}], 'movement_move': [{'tarjetasMovidas': 3}], 'text': [{'tarjetasMovidas': 2}], 'movement_activate_door': [{'tarjetasMovidas': 1}]}","{'tarjetasCreadas': 0, 'variables_get': [{'tarjetasMovidas': 6}], 'variables_set': [{'tarjetasMovidas': 6}], 'movement_move_laser': [{'tarjetasMovidas': 5}], 'movement_laser_change_intensity': [{'tarjetasMovidas': 1}], 'text': [{'tarjetasMovidas': 3}], 'math_number': [{'tarjetasMovidas': 2}], 'movement_rotate': [{'tarjetasMovidas': 2}], 'start_start': [{'tarjetasMovidas': 3}], 'movement_move': [{'tarjetasMovidas': 3}], 'movement_activate_door': [{'tarjetasMovidas': 1}], 'logic_compare': [{'tarjetasMovidas': 1}], 'math_arithmetic': [{'tarjetasMovidas': 1}], 'logic_cells_occupied': [{'tarjetasMovidas': 1}]}","{'tarjetasCreadas': 0, 'variables_get': [{'tarjetasMovidas': 13}], 'variables_set': [{'tarjetasMovidas': 9}], 'movement_move_laser': [{'tarjetasMovidas': 1}], 'math_number': [{'tarjetasMovidas': 4}], 'text': [{'tarjetasMovidas': 3}], 'movement_move': [{'tarjetasMovidas': 2}], 'math_arithmetic': [{'tarjetasMovidas': 1}], 'logic_cells_occupied': [{'tarjetasMovidas': 2}], 'logic_negate': [{'tarjetasMovidas': 3}], 'movement_activate_door': [{'tarjetasMovidas': 2}]}","{'tarjetasCreadas': 0, 'variables_get': [{'tarjetasMovidas': 31}], 'variables_set': [{'tarjetasMovidas': 35}], 'math_number': [{'tarjetasMovidas': 9}], 'movement_move_laser': [{'tarjetasMovidas': 2}], 'text': [{'tarjetasMovidas': 7}]}","{'tarjetasCreadas': 0, 'movement_move_laser': [{'tarjetasMovidas': 2}], 'variables_set': [{'tarjetasMovidas': 9}], 'variables_get': [{'tarjetasMovidas': 5}], 'movement_rotate_laser': [{'tarjetasMovidas': 1}], 'text': [{'tarjetasMovidas': 1}], 'math_number': [{'tarjetasMovidas': 4}], 'movement_rotate': [{'tarjetasMovidas': 1}]}","{'tarjetasCre